In [6]:
import numpy as np
from qiskit import ClassicalRegister, QuantumRegister,QuantumCircuit,execute,Aer
from qiskit.tools.visualization import circuit_drawer,plot_histogram

# Set the size circuit
qftsize = 4 #x qubits number
nbits = 4   #y qubits number
#Real SHOR => csize= qftsize 
csize= qftsize + nbits # classical register number

xr = QuantumRegister(qftsize, 'x')
yr = QuantumRegister(nbits,'y')
cr = ClassicalRegister(csize, 'c')

circuit = QuantumCircuit(xr,yr, cr)
# Apply Hadamart gate

for i in range(qftsize):
    circuit.h(xr[i])
circuit.barrier() # separate

# 7 times  controlled circuit

circuit.x(yr[0])
circuit.cx(xr[0],yr[1])
circuit.cx(xr[0],yr[2])
circuit.barrier() # separate


# 4 times mode 15 circuit 7^2 mod 15 = 4
# 4Y mod 15 = 4 y0+ 8y1 +y2 + 2y3.  ( y0 y1 y2 y3) (y1 y0 y3 y2)
# Using Fredkin Gate 

circuit.cswap(xr[1],yr[0],yr[2])
circuit.cswap(xr[1],yr[1],yr[3])
circuit.barrier() #separate

# times a^4 mod N
# 7^4 mod 15 = 1 (no action required)
# Same for a^8

# Measurement

for i in range(qftsize):
    circuit.measure(xr[i], cr[i])
    
# a^x mod N calculation measurement

for i in range(nbits):
    circuit.measure(yr[i],cr[i+qftsize])
    
    
#シミュレーターの指定, select the simulator
simulator = Aer.get_backend('qasm_simulator')
#量子回路のシミュレーターを使った実行, Repeat circuit= #(shots) times
job = execute(circuit, simulator, shots=1000)

#測定結果の取り出し
result = job.result()
counts = result.get_counts(circuit)



In [7]:
# Show results counts

print("Total count [y3,y2,y1,y0,x3,x2,x1,x0]:", counts )

Total count [y3,y2,y1,y0,x3,x2,x1,x0]: {'01111101': 76, '11011111': 57, '00010100': 58, '00011100': 71, '00010000': 61, '01000110': 75, '00011000': 71, '01111001': 49, '01110001': 60, '01110101': 64, '11010011': 44, '11010111': 47, '11011011': 70, '01001010': 71, '01001110': 64, '01000010': 62}


In [8]:
# Convert to (X,Y)  and verify that Y= 7^X mod 15

scounts = sorted(counts.items(), key= lambda x:x[0][4:8]) # sort xbits

for i in scounts:
    k =i[0]
    v= i[1]
    ybits = k[0:4]  #get the y bits
    xbits = k[4:8]  # get the x bits
    print('(X,Y)= ', xbits,ybits, "=(", int(xbits,2), ',', int(ybits,2),'): count', v )
    
# period of 4


(X,Y)=  0000 0001 =( 0 , 1 ): count 61
(X,Y)=  0001 0111 =( 1 , 7 ): count 60
(X,Y)=  0010 0100 =( 2 , 4 ): count 62
(X,Y)=  0011 1101 =( 3 , 13 ): count 44
(X,Y)=  0100 0001 =( 4 , 1 ): count 58
(X,Y)=  0101 0111 =( 5 , 7 ): count 64
(X,Y)=  0110 0100 =( 6 , 4 ): count 75
(X,Y)=  0111 1101 =( 7 , 13 ): count 47
(X,Y)=  1000 0001 =( 8 , 1 ): count 71
(X,Y)=  1001 0111 =( 9 , 7 ): count 49
(X,Y)=  1010 0100 =( 10 , 4 ): count 71
(X,Y)=  1011 1101 =( 11 , 13 ): count 70
(X,Y)=  1100 0001 =( 12 , 1 ): count 71
(X,Y)=  1101 0111 =( 13 , 7 ): count 76
(X,Y)=  1110 0100 =( 14 , 4 ): count 64
(X,Y)=  1111 1101 =( 15 , 13 ): count 57
